### This is the basic idea and code snippets of our MWPM decoder.
first need to set some parameter for algorithm

In [ ]:
p_x = 0.05
p_y = 0.05
p_z = 0.05
local_search_m = 25 # the maximun degree when apply local dijkstra to build syndrome graph

#### 1. Read the testpoint

In [1]:
import numpy as np

# error_x, error_z, syndrome_x, syndrome_z,sizeX,sizeY = read_test_file('test_file.npz')
def read_test_file(filename):
    file_path = "../code/data/input/" + filename
    data = np.load(file_path, allow_pickle=True)
    file_content = data['arr_0'].item()
    return file_content["sizeX"], file_content["sizeY"] ,file_content["syndrome_x"], file_content["syndrome_z"]

def write_result_file(consume_time,correct_x,correct_z,filename):
    file_content = {
        "time": consume_time,
        "correct_x":correct_x,
        "correct_z":correct_z,
    }
    np.savez("../code/data/output/"+filename, file_content)
    return file_content

sizeX,sizeY,syndrome_x,syndrome_z = read_test_file('test1.npz')

#### 2. Build Matching graph
apply weight to every qubit like this:
$
w_i = \log\left(\frac{1 - p_i}{p_i}\right)
$

where $p_i$ is the physical error probability of ith qubit. So here generate all weight first.


In [ ]:
# generate weight matrix for every qubit, the error rate of Z
weight_mat = np.full((sizeY*2,sizeX),p_z+p_y)

#### 3. Build Syndrome graph

using dijkstra / local dijkstra to build a syndrome graph


because we will use dijkstra algorithm also in blossom algorithm, so build linked_list for syndrome graph

#### 4. Blossom
use blossom algorithm to find perfect matching

#### 5. output correct graph
Need to output every edge(sizeX\*sizeY\*2) on the matching graph, tell whether we detect error there.